In [ ]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, stemming, tfidf_vectorizer, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [3]:
#define model and pipeline
bidirLSTM = BidirectionalLSTM(vocab_size=10000, embedding_dim=100, max_sequence_length=100, lstm_units=128)
bidirLSTM.set_pipeline([clean_text, stop_words_removal])

#import dataset and apply pipeline
inputs, targets = get_dataset()
inputs = bidirLSTM.run_pipeline(inputs, save=False)

print(inputs[0:5])
print(targets)

#stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

[[<function clean_text at 0x00000283DBFCD300>, <function stop_words_removal at 0x00000283DBFCD1C0>]]
Pipeline execution time: 0:00:18.191163
['23 funniest tweets cats dogs week sept 1723 dog dont understand could eaten', 'amazon greenlights blade runner 2099 limited series produced ridley scott director original 1982 film joins writer 2017 sequel newest installment scifi franchise', 'emmy awards viewership dips recordlow audience continues drop television awards ceremony lost roughly 15 million viewers compared 2021 program', 'nod jfk joe biden pushing moonshot fight cancer biden plans draw attention new federally backed study seeks validate using blood tests screen multiple cancers', 'politicians dna connected las vegas journalists murder police say clark county public administrator robert telles arrested fatal stabbing jeff german investigating office']
2         Entertainment
17               Sports
20        Entertainment
21             Politics
24             Politics
            

In [ ]:
bidirLSTM.fit(x_train, y_train)

In [ ]:
#model predictions

y_pred = bidirLSTM.predict(x_test)

results = bidirLSTM.compute_performance(y_test, y_pred)
bidirLSTM.print_performance(results)
